In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [10]:
from keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop

In [ ]:
img_size = (48, 48)
folder_path = "images/"

expression = 'happy'

plt.figure(figsize= (12,12))
for i in range(1, 10, 1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"train/"+expression+"/"+os.listdir(folder_path + "train/" + expression)[i], target_size=img_size)
    plt.imshow(img)   
plt.show()

In [11]:
batch_size=128

datagen_train = ImageDataGenerator(1./255)
datagen_validation = ImageDataGenerator(1./255)

train_set= datagen_train.flow_from_directory(
    "images/train",
    class_mode="categorical",
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    shuffle=True
)

test_set= datagen_validation.flow_from_directory(
    "images/validation",
    class_mode="categorical",
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    shuffle=False
)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [12]:
print(train_set)

<BatchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>


In [7]:
no_of_classes=7

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding="same",input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


#2nd CNN layer
model.add(Conv2D(128,(5,5),padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation="softmax"))

model.compile(optimizer=Adam(lr = 0.0001) ,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

c:\Users\KANISHK\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [31]:
model.save("./fer_model.h5")

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

checkpoint = ModelCheckpoint("./model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                                min_delta=0,
                                patience=3,
                                verbose=1,
                                restore_best_weights=True
                                )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.2,
                                        patience=3,
                                        verbose=1,
                                        min_delta=0.0001)

callbacks_list = [early_stopping, checkpoint, reduce_learningrate]

epochs = 20

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])


with tf.device('/GPU:0'):
    history = model.fit(train_set,
                        steps_per_epoch=train_set.n//train_set.batch_size,
                        epochs=epochs,
                        validation_data = test_set,
                        validation_steps = test_set.n//test_set.batch_size,
                        callbacks=callbacks_list
                        )

c:\Users\KANISHK\miniconda3\envs\tf\lib\site-packages\keras\preprocessing\image.py:1863: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/15
225/225 [==============================] - 62s 268ms/step - loss: 1.7874 - accuracy: 0.3077 - val_loss: 1.6206 - val_accuracy: 0.3771 - lr: 0.0010
Epoch 2/15
225/225 [==============================] - 53s 236ms/step - loss: 1.4479 - accuracy: 0.4414 - val_loss: 1.5457 - val_accuracy: 0.4339 - lr: 0.0010
Epoch 3/15
225/225 [==============================] - 62s 277ms/step - loss: 1.2869 - accuracy: 0.5080 - val_loss: 1.4287 - val_accuracy: 0.4688 - lr: 0.0010
Epoch 4/15
225/225 [==============================] - 54s 240ms/step - loss: 1.1991 - accuracy: 0.5388 - val_loss: 1.2336 - val_accuracy: 0.5286 - lr: 0.0010
Epoch 5/15
225/225 [==============================] - 55s 244ms/step - loss: 1.1323 - accuracy: 0.5681 - val_loss: 1.2507 - val_accuracy: 0.5455 - lr: 0.0010
Epoch 6/15
225/225 [==============================] - 55s 243ms/step - loss: 1.0812 - accuracy: 0.5908 - val_loss: 1.1300 - val_accuracy: 0.5803 - lr: 0.0010
Epoch 7/15
225/225 [==============================] 

In [30]:
model.save_weights("./face_emotion_model.h5")

In [25]:
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
# emotions[np.argmax(model.predict(np.array([img_to_array(load_img('images/train/angry/0.jpg', color_mode="grayscale"))])))]
def predict_emotion(image):
	# return emotions[np.argmax(model.predict(np.array([img_to_array(image)])))]
	return emotions[np.argmax(model.predict(np.asarray(image)))]


1/1 [==============================] - 0s 83ms/step
happy


In [29]:
print(predict_emotion(np.array([img_to_array(load_img('images/train/happy/64.jpg', color_mode="grayscale"))])))

1/1 [==============================] - 0s 36ms/step
happy
